This script is used to generate the frequency, total days, and intensity of the UHWs using **CMIP grid cell** maximal tempeature

The data sets are from:   
```bash
/data/keeling/a/zzheng25/d/CMIP5_tasmax_csv/
```

The results are saved at:
```
S3://uhws/RHWs_CMIP/
```

Note:     
**2006**: Using 2006 itself to calculate the percentile, frequency, total days, and intensity  
**2061**: Using the percentile of **2006** to calculate frequency, total days, and intensity of 2061   

In [1]:
import xarray as xr
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import gc
import util
from s3fs.core import S3FileSystem
s3 = S3FileSystem()

save_dir = "S3://uhws/RHWs_CMIP/"

In [2]:
CMIP5_ls = ["ACCESS1-0", "ACCESS1-3", "CanESM2", "CNRM-CM5", "CSIRO-Mk3-6-0",
            "FGOALS-s2","GFDL-CM3", "GFDL-ESM2G", "GFDL-ESM2M", "HadGEM2-CC",
            "HadGEM2-ES", "IPSL-CM5A-MR", "MIROC5", "MIROC-ESM", "MIROC-ESM-CHEM",
            "MRI-CGCM3", "MRI-ESM1"]

## Step 1: Start the pipeline to use 98% percentile (2006) to get frequency (events/year), total days (days/year), and intensity (K) of 2006 and 2061

In [3]:
frequency_2006_ls=[]
duration_2006_ls=[]
intensity_2006_ls=[]
quantile_avail_2006_ls=[]

frequency_2061_ls=[]
duration_2061_ls=[]
intensity_2061_ls=[]

for model in CMIP5_ls:
    print("start model:",model)
    
    # start 2006
    start_time_2006=time.time()
    
    df_2006=util.load_df("/data/keeling/a/zzheng25/d/CMIP5_tasmax_csv/"+model+"/2006.csv")
    cmip_2006_hw, quantile_avail_2006=util.get_heat_waves_df(df_2006, 0.98, 2, "cmip", None)
    
    frequency_2006_ls.append(util.get_frequency(cmip_2006_hw,model))
    duration_2006_ls.append(util.get_duration(cmip_2006_hw,model))
    intensity_2006_ls.append(util.get_intensity(cmip_2006_hw,model))
    quantile_avail_2006_ls.append(quantile_avail_2006.copy().rename(columns={"quant": model}).set_index(["lat","lon"]))
    print("It took",time.time()-start_time_2006,"to deal with",model,"for year 2006")

    
    # start 2061
    start_time_2061=time.time()
    df_2061=util.load_df("/data/keeling/a/zzheng25/d/CMIP5_tasmax_csv/"+model+"/2061.csv")
    cmip_2061_hw, quantile_avail_2061=util.get_heat_waves_df(df_2061, None, 2, "cmip", quantile_avail_2006)
    
    frequency_2061_ls.append(util.get_frequency(cmip_2061_hw,model))
    duration_2061_ls.append(util.get_duration(cmip_2061_hw,model))
    intensity_2061_ls.append(util.get_intensity(cmip_2061_hw,model))
    
    print("It took",time.time()-start_time_2061,"to deal with",model,"for year 2061")
    print("\n")
    
    del df_2006, df_2061, quantile_avail_2006, quantile_avail_2061
    gc.collect()

start model: ACCESS1-0
Start to load csv /data/keeling/a/zzheng25/d/CMIP5_tasmax_csv/ACCESS1-0/2006.csv
It takes 10.62881851196289 to load csv
The quantile is: 0.98
The duration threshold is: 2


/data/keeling/a/zzheng25/UrbanHWs/event_analysis/v4/util.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_quantile["HW"][df_with_quantile["mean"]> df_with_quantile["quant"]] = 0


It took 36.8468816280365 to deal with ACCESS1-0 for year 2006
Start to load csv /data/keeling/a/zzheng25/d/CMIP5_tasmax_csv/ACCESS1-0/2061.csv
It takes 11.019837141036987 to load csv
The quantile is: None
The duration threshold is: 2
It took 32.916799545288086 to deal with ACCESS1-0 for year 2061


start model: ACCESS1-3
Start to load csv /data/keeling/a/zzheng25/d/CMIP5_tasmax_csv/ACCESS1-3/2006.csv
It takes 10.401712417602539 to load csv
The quantile is: 0.98
The duration threshold is: 2
It took 38.41004157066345 to deal with ACCESS1-3 for year 2006
Start to load csv /data/keeling/a/zzheng25/d/CMIP5_tasmax_csv/ACCESS1-3/2061.csv
It takes 10.443950176239014 to load csv
The quantile is: None
The duration threshold is: 2
It took 33.87665820121765 to deal with ACCESS1-3 for year 2061


start model: CanESM2
Start to load csv /data/keeling/a/zzheng25/d/CMIP5_tasmax_csv/CanESM2/2006.csv
It takes 10.212414979934692 to load csv
The quantile is: 0.98
The duration threshold is: 2
It took 36.426

In [4]:
frequency_2006 = pd.concat(frequency_2006_ls, axis=1)
duration_2006 = pd.concat(duration_2006_ls, axis=1)
intensity_2006 = pd.concat(intensity_2006_ls, axis=1)
quantile_avail_2006 = pd.concat(quantile_avail_2006_ls, axis=1)

frequency_2061 = pd.concat(frequency_2061_ls, axis=1)
duration_2061 = pd.concat(duration_2061_ls, axis=1)
intensity_2061 = pd.concat(intensity_2061_ls, axis=1)

In [5]:
# here the quantile 2006 and quantile 2061 should be same
frequency_2006.to_csv(save_dir+"2006_frequency.csv")
duration_2006.to_csv(save_dir+"2006_totaldays.csv")
intensity_2006.to_csv(save_dir+"2006_intensity.csv")
quantile_avail_2006.to_csv(save_dir+"2006_percentile.csv")

frequency_2061.to_csv(save_dir+"2061_frequency.csv")
duration_2061.to_csv(save_dir+"2061_totaldays.csv")
intensity_2061.to_csv(save_dir+"2061_intensity.csv")